## Examples

In [2]:
!python -m pip install numpy

In [3]:
!python -m pip uninstall -y changepoint_online

Found existing installation: changepoint_online 0.0.3
Uninstalling changepoint_online-0.0.3:
  Successfully uninstalled changepoint_online-0.0.3


In [4]:
!python -m pip install '../package/'

Processing /home/romano/Documents/changepoint.online/python/package
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for changepoint_online: filename=changepoint_online-0.0.3-py3-none-any.whl size=20738 sha256=bacd6d8b7fea0b8d6459e905c2293334c2396e2f4fc8fce4ffcf8bd79ba97d92
  Stored in directory: /tmp/pip-ephem-wheel-cache-3jaetfjf/wheels/5d/82/f0/6cc0febe5e7138cd795bfda4f32fc28c65e661d55018ebc214
Successfully built changepoint_online


In [5]:
from changepoint_online import *

In [6]:
help(Focus)

Help on class Focus in module changepoint_online.focus:

class Focus(builtins.object)
 |  Focus(comp_func, side='both')
 |  
 |  The Focus class implements the Focus method, an algorithm for detecting changes in data streams on one-parameter exponential family models.
 |  For instance, Focus can detect changes-in-mean in a Gaussian data stream (white noise). 
 |  It can be applied to settings where either the pre-change parameter is known or unknown. Furthremore, the test can be 
 |  constrained to detect either an increase or a decrease in the value of the parameter.
 |      
 |  Focus solves the CUSUM likelihood-ratio test exactly in O(log(n)) time per iteration, where n is the current iteration. 
 |  The method is equivalent to running a rolling window (MOSUM) simultaneously for all sizes of window, or the Page-CUSUM for all possible values
 |  of the size of change (an infinitely dense grid). 
 |  
 |  DISCLAIMER: Albeit the Focus algorithm decreases the per-iteration cost from O(n

In [7]:
import numpy as np

In [8]:
np.random.seed(0)
Y = np.concatenate((np.random.normal(loc=0.0, scale=1.0, size=5000), np.random.normal(loc=10.0, scale=1.0, size=5000)))

### Guassian

In [9]:
detector = Focus(Gaussian(loc=0))

In [10]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [11]:
detector.changepoint()

{'stopping_time': 5001, 'changepoint': 5000}

### Poisson

In [12]:
np.random.seed(0)
Y = np.concatenate((np.random.poisson(lam=4.0, size=5000), np.random.poisson(lam=4.2, size=5000)))

In [13]:
detector = Focus(Poisson())

In [14]:
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break

In [15]:
detector.changepoint()

{'stopping_time': 6488, 'changepoint': 4973}

### Gamma

In [16]:
np.random.seed(0)
Y = np.concatenate((np.random.gamma(4.0, scale=3.0, size=5000), np.random.gamma(4.0, scale=6.0, size=5000)))

In [17]:
detector = Focus(Gamma(scale=3.0, shape=4.0))
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

{'stopping_time': 5010, 'changepoint': 5002}

In [18]:
# with the Gamma it's possible to initialize even with rate parameter
detector = Focus(Gamma(rate=1/3.0, shape=4.0))
threshold = 10.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

{'stopping_time': 5010, 'changepoint': 5002}

### Bernoulli

In [19]:
np.random.seed(0)
Y = np.concatenate((np.random.binomial(n=1, p=.4, size=5000), np.random.binomial(n=1, p=.8, size=5000)))

In [20]:
detector = Focus(Bernoulli())
threshold = 15.0
for y in Y:
    detector.update(y)
    if detector.statistic() >= threshold:
        break
detector.changepoint()

{'stopping_time': 5024, 'changepoint': 4999}

NP FOCus

In [29]:
import numpy as np

# Define a simple Gaussian noise function
def generate_gaussian_noise(size):
    return np.random.normal(loc=0.0, scale=1.0, size=size)

# Generate mixed data with change in gamma component
gamma_1 = np.random.gamma(4.0, scale=3.0, size=5000)
gamma_2 = np.random.gamma(4.0, scale=6.0, size=5000)
gaussian_noise = generate_gaussian_noise(10000)
Y = np.concatenate((gamma_1 + gaussian_noise[:5000], gamma_2 + gaussian_noise[5000:]))

# Create and use NPFocus detector
quantiles = [np.quantile(Y[:100], q) for q in [0.25, 0.5, 0.75]]
detector = NPFocus(quantiles)

stat_over_time = []

for y in Y:
    detector.update(y)
    if np.sum(detector.statistic()) > 20:
        break


changepoint_info = detector.changepoint()

print(f"Changepoint information:\n{changepoint_info}")

Changepoint information:
{'stopping_time': 5018, 'changepoint': 5011, 'max_stat': 9.039371293991735}


[[0.0, 0.0, 0.0],
 [1.3862943591198906, 0.0, 0.0],
 [0.5232481427645477, 1.9095425018844383, 0.0],
 [0.8630462153553426, 2.7725887182397813, 2.2493405744752333],
 [1.1157177535710487, 3.365058330046282, 3.365058330046282],
 [1.3170728880779377, 3.8190850037688766, 1.9095425018844383],
 [1.4845198618989721, 2.2783446132123624, 1.41529839485702],
 [1.6278669229316036, 2.5891386500660287, 1.927447567217575],
 [1.7531944944673974, 2.857813284634452, 2.3635691766070335],
 [1.8645353647945917, 2.231435511142098, 1.9497599341892622],
 [1.9647027007217293, 1.850474106589929, 1.6651220654357575],
 [2.0557374345540365, 2.5891386490660278, 1.4555158271618431],
 [0.8439078531365007, 1.762114273329134, 1.8414888131185307],
 [2.013578172101158, 1.5537413339409252, 2.1931200458176923],
 [1.3824505599914163, 1.7022298863164824, 2.516073375895602],
 [2.281306908013031, 1.5296414211525207, 2.814708117525023],
 [1.7652276032748428, 1.658898312352168, 1.3011951744282335],
 [1.4033577792928593, 1.511621931